In [1]:
#Set up
import pandas as pd 
import numpy as np


input_df = pd.read_csv('../data/train/input_2023_w01.csv')
print(input_df.columns)


output_df = pd.read_csv('../data/train/output_2023_w01.csv')

Index(['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id',
       'play_direction', 'absolute_yardline_number', 'player_name',
       'player_height', 'player_weight', 'player_birth_date',
       'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a',
       'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y'],
      dtype='object')
